# Networking / tcpdump tutorial

This tutorial will look at several networking tools available in the default Debian 'Jessie' repositories, especially tcpdump

### What tcpdump version are we running in this notebook?

First let's check what version of tcpdump we are running - in the interests of documenting our experiment

In [14]:
tcpdump --version

tcpdump version 4.6.2
libpcap version 1.6.2
OpenSSL 1.0.1k 8 Jan 2015


### Capturing with tcpdump

As we don't have permission to listen on network interfaces we will work with existing pcap capture files, already created with tcpdump.

Nevertheless, we first look at how to capture your own pcap files with tcpdump

On a linux system, such as Ubuntu, run
```/sbin/ip a```
command to see what network interfaces you have.

```
1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN group default qlen 1
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
    inet 127.0.0.1/8 scope host lo
       valid_lft forever preferred_lft forever
    inet6 ::1/128 scope host
       valid_lft forever preferred_lft forever
2: enp3s0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc pfifo_fast state UP group default qlen 1000
    link/ether 40:8d:5c:a0:a2:c8 brd ff:ff:ff:ff:ff:ff
    inet 192.168.0.34/24 brd 192.168.0.255 scope global dynamic enp3s0
       valid_lft 34763sec preferred_lft 34763sec
    inet6 fe80::30a7:9710:ffa6:eb21/64 scope link
       valid_lft forever preferred_lft forever
3: wlp2s0: <NO-CARRIER,BROADCAST,MULTICAST,UP> mtu 1500 qdisc mq state DOWN group default qlen 1000
    link/ether b4:6d:83:d2:a2:f3 brd ff:ff:ff:ff:ff:ff
```

We see that my system has 2 networking interfaces
- a wired connection 'enp3s0'
- a wireless connection 'wlp2so'.

So to sniff for traffic on my wired connection I would enter the command

```tcpdump -i enp3s0```

**Note:** you will need to run this command as root, or under sudo

### capturing to a file

``` tcpdump -i enp3s0 -w sample.pcap```

Here is an example of capturing a pcap file on my system.
We can terminate the capture by interrupting by typing ctrl-C, or we can specify the maximum number of packets by adding the `*-c <count>*' parameters


#### capturing until interruption:
<pre>
tcpdump: listening on enp3s0, link-type EN10MB (Ethernet), capture size 262144 bytes
10 packets captured
12 packets received by filter
0 packets dropped by kernel
mjb@NUC:~$ ll sample.pcap
-rw-r--r-- 1 root root 1126 Jul  9 08:12 sample.pcap
</pre>

#### capturing 100 packets, using
``` tcpdump -v -i enp3s0 -c 100 -w sample100.pcap```
Note: we add -v option so that we see the number of packets captured whilst waiting for completion

<pre>
tcpdump: listening on enp3s0, link-type EN10MB (Ethernet), capture size 262144 bytes
100 packets captured
100 packets received by filter
0 packets dropped by kernel
mjb@NUC:~$ ll sample100.pcap
-rw-r--r-- 1 root root 10172 Jul  9 08:23 sample100.pcap
</pre>



## Analyzing packets

We could add many parameters to the above capture commands, but we will investigate their effects whilst reading pre-captured pcap files

In [3]:
ls dhcpProblem.capture.pcap

dhcpProblem.capture.pcap


In [4]:
ls -altr dhcpProblem.capture.pcap

-rw-r--r-- 1 main main 3684 Jul  9 05:53 dhcpProblem.capture.pcap


In [13]:
# will do dns lookup (slow!)
time tcpdump -r dhcpProblem.capture.pcap

reading from file dhcpProblem.capture.pcap, link-type EN10MB (Ethernet)
18:11:26.385144 IP6 :: > ff02::16: HBH ICMP6, multicast listener report v2, 1 group record(s), length 28
18:11:26.463826 IP 0.0.0.0.bootpc > 255.255.255.255.bootps: BOOTP/DHCP, Request from fa:16:3e:01:85:d1 (oui Unknown), length 280
18:11:26.464311 IP 10.0.0.1.bootps > 10.0.0.4.bootpc: BOOTP/DHCP, Reply, length 316
18:11:26.545760 IP6 :: > ff02::1:ff01:85d1: ICMP6, neighbor solicitation, who has fe80::f816:3eff:fe01:85d1, length 24
18:11:26.585544 IP6 :: > ff02::16: HBH ICMP6, multicast listener report v2, 1 group record(s), length 28
18:11:29.485856 IP 0.0.0.0.bootpc > 255.255.255.255.bootps: BOOTP/DHCP, Request from fa:16:3e:01:85:d1 (oui Unknown), length 280
18:11:29.486185 IP 10.0.0.1.bootps > 10.0.0.4.bootpc: BOOTP/DHCP, Reply, length 316
18:11:32.506631 IP 0.0.0.0.bootpc > 255.255.255.255.bootps: BOOTP/DHCP, Request from fa:16:3e:01:85:d1 (oui Unknown), length 280
18:11:32.507379 IP 10.0.0.1.bootps > 10.0.0.

In [12]:
time tcpdump -n -r dhcpProblem.capture.pcap

reading from file dhcpProblem.capture.pcap, link-type EN10MB (Ethernet)
18:11:26.385144 IP6 :: > ff02::16: HBH ICMP6, multicast listener report v2, 1 group record(s), length 28
18:11:26.463826 IP 0.0.0.0.68 > 255.255.255.255.67: BOOTP/DHCP, Request from fa:16:3e:01:85:d1, length 280
18:11:26.464311 IP 10.0.0.1.67 > 10.0.0.4.68: BOOTP/DHCP, Reply, length 316
18:11:26.545760 IP6 :: > ff02::1:ff01:85d1: ICMP6, neighbor solicitation, who has fe80::f816:3eff:fe01:85d1, length 24
18:11:26.585544 IP6 :: > ff02::16: HBH ICMP6, multicast listener report v2, 1 group record(s), length 28
18:11:29.485856 IP 0.0.0.0.68 > 255.255.255.255.67: BOOTP/DHCP, Request from fa:16:3e:01:85:d1, length 280
18:11:29.486185 IP 10.0.0.1.67 > 10.0.0.4.68: BOOTP/DHCP, Reply, length 316
18:11:32.506631 IP 0.0.0.0.68 > 255.255.255.255.67: BOOTP/DHCP, Request from fa:16:3e:01:85:d1, length 280
18:11:32.507379 IP 10.0.0.1.67 > 10.0.0.4.68: BOOTP/DHCP, Reply, length 316
18:11:37.516460 ARP, Request who-has 10.0.0.4 tell

In [9]:
tcpdump -qns 0 -A -r dhcpProblem.capture.pcap

reading from file dhcpProblem.capture.pcap, link-type EN10MB (Ethernet)
18:11:26.385144 IP6 :: > ff02::16: HBH ICMP6, multicast listener report v2, 1 group record(s), length 28
`....$..................................:...................................
18:11:26.463826 IP 0.0.0.0.68 > 255.255.255.255.67: UDP, length 280
E..4....@.y..........D.C. BZ......E3......................>.............................................................................................................................................................................................................c.Sc5..=....>...9..@7.......*<.udhcp 1.18.5.
18:11:26.464311 IP 10.0.0.1.67 > 10.0.0.4.68: UDP, length 316
E..X....@.Z.
...
....C.D.D.Z......E3........
...
.........>.............................................................................................................................................................................................................c.Sc5..6.
...3....x:....<;....i........
..?

In [6]:
man tcpdump

bash: man: command not found


In [7]:
tcpdump --help

tcpdump version 4.6.2
libpcap version 1.6.2
OpenSSL 1.0.1k 8 Jan 2015
Usage: tcpdump [-aAbdDefhHIJKlLnNOpqRStuUvxX#] [ -B size ] [ -c count ]
		[ -C file_size ] [ -E algo:secret ] [ -F file ] [ -G seconds ]
		[ -i interface ] [ -j tstamptype ] [ -M secret ] [ --number ]
		[ -Q in|out|inout ]
		[ -r file ] [ -s snaplen ] [ --time-stamp-precision precision ]
		[ -T type ] [ --version ] [ -V file ]
		[ -w file ] [ -W filecount ] [ -y datalinktype ] [ -z command ]
		[ -Z user ] [ expression ]
